Bibliotecas

In [20]:
import os
import numpy as np
import pandas as pd

Lista arquivos do diretório

In [21]:
def listar_diretorio(diretorio, extensao):
    arquivos_diretorio = []

    for arquivo in os.listdir(diretorio):
        if arquivo.endswith(extensao):
           arquivos_diretorio.append(arquivo)

    return arquivos_diretorio

Análise de Estrutura dos Dados

In [23]:
def analisar_estrutura_dados(diretorio, arquivos_csv):
    df_dados          = pd.DataFrame()    
    df_estrutra_dados = pd.DataFrame()

    for arquivo in arquivos_csv:
        caminho_arquivo   = os.path.join(diretorio, arquivo)
        df_dados          = pd.read_csv(caminho_arquivo, encoding="ISO-8859-1", sep=';', low_memory=False)
        df_estrutra_dados = pd.concat([df_estrutra_dados, df_dados], ignore_index=True)

    print("Qtde de registros")
    print("-----------------")
    print("")
    print(len(df_estrutra_dados.axes[0]))    
    
    print("")
    print("") 
    print("Análise da estrutura")
    print("--------------------")
    print("")
    df_estrutra_dados.info()

    print("")
    print("") 
    print("Qtde de campos nulos")
    print("--------------------")
    print("")
    print(df_estrutra_dados.isnull().sum())

Carregamento dos datasets

In [25]:
def carregar_arquivos(diretorio, lista_arquivos_csv):
    df_dados              = pd.DataFrame()
    df_dados_ano_semestre = pd.DataFrame()
   
    tipos_de_dados = {'Valor de Venda': np.float64,
                      'Valor de Compra': np.float64}

    for arquivo in lista_arquivos_csv:
        caminho_arquivo = os.path.join(diretorio, arquivo)
        df_dados        = pd.read_csv(caminho_arquivo, encoding="ISO-8859-1", 
                                      sep=';', thousands = '.', decimal = ',', 
                                      dtype = {'Regiao - Sigla': 'category',
                                               'Estado - Sigla': 'category',
                                               'Valor de Venda': np.float64,
                                               'Valor de Compra': np.float64,
                                               'Regiao - Sigla': 'category'})
        
        # Extrai informações do nome do arquivo
        partes_nome = arquivo.split('-')
        ano         = partes_nome[1]
        semestre    = partes_nome[2].split('.')[0]

        # Adiciona informações de ano e semestre como colunas
        df_dados['Ano']      = ano
        df_dados['Semestre'] = semestre

        df_dados['Ano']      = df_dados['Ano'].astype(np.int16)
        df_dados['Semestre'] = df_dados['Semestre'].astype(np.int8)
        
        df_dados_ano_semestre = pd.concat([df_dados_ano_semestre, df_dados], ignore_index=True)
        
    return df_dados_ano_semestre

Chamada para a Análise da estrutura dos dados

In [26]:
diretorio  = 'datasets'

arquivos_csv = []
arquivos_csv = listar_diretorio(diretorio, '.csv')

df_dados = pd.DataFrame()
df_dados = analisar_estrutura_dados(diretorio, arquivos_csv)

Qtde de registros
-----------------

10265783


Análise da estrutura
--------------------

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10265783 entries, 0 to 10265782
Data columns (total 17 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   ï»¿Regiao - Sigla  object
 1   Estado - Sigla     object
 2   Municipio          object
 3   Revenda            object
 4   CNPJ da Revenda    object
 5   Nome da Rua        object
 6   Numero Rua         object
 7   Complemento        object
 8   Bairro             object
 9   Cep                object
 10  Produto            object
 11  Data da Coleta     object
 12  Valor de Venda     object
 13  Valor de Compra    object
 14  Unidade de Medida  object
 15  Bandeira           object
 16  Regiao - Sigla     object
dtypes: object(17)
memory usage: 1.3+ GB


Qtde de campos nulos
--------------------

ï»¿Regiao - Sigla     472856
Estado - Sigla             0
Municipio                  0
Revenda                    0
CNP

Principal

In [28]:
diretorio  = 'datasets'

lista_arquivos_csv = []
lista_arquivos_csv = listar_diretorio(diretorio, '.csv')

df_dados = pd.DataFrame()
df_dados = carregar_arquivos(diretorio, lista_arquivos_csv)

print(df_dados)



         ï»¿Regiao - Sigla Estado - Sigla  Municipio  \
0                       SE             SP  GUARULHOS   
1                       SE             SP  GUARULHOS   
2                       SE             SP  GUARULHOS   
3                       SE             SP  GUARULHOS   
4                       CO             DF   BRASILIA   
...                    ...            ...        ...   
10265778                 S             RS    LAJEADO   
10265779                 S             RS    LAJEADO   
10265780                 S             RS    LAJEADO   
10265781                 S             RS    LAJEADO   
10265782                 S             RS    LAJEADO   

                                        Revenda      CNPJ da Revenda  \
0                      AUTO POSTO SAKAMOTO LTDA   49.051.667/0001-02   
1                      AUTO POSTO SAKAMOTO LTDA   49.051.667/0001-02   
2                      AUTO POSTO SAKAMOTO LTDA   49.051.667/0001-02   
3                      AUTO POSTO SAKAM